<a href="https://colab.research.google.com/github/clementbowe14/ml-class/blob/main/projects/AmazonReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


model_scores = []
data_df = pd.read_csv("http://34.82.108.37/amazonReviewsTrain.csv.zip")
data_df.head()

<ipython-input-78-c2d660663bad>:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv("http://34.82.108.37/amazonReviewsTrain.csv.zip")


,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image,rating
0,True,"05 4, 2017",A2QNLVLQHW6OOC,B00NEZNT74,Jason L.,Have 30 of these in operation great lights,Five Stars,1493856000,NaN,{'Size:': ' 4PC'},NaN,positive
1,True,"12 1, 2014",A32SHV461SUWNQ,B002Q0WTCE,Richard C.,"Good replacement. I prefer my Rovner L5, but ...",Good replacement. I prefer my Rovner L5,1417392000,NaN,NaN,NaN,positive
2,True,"12 23, 2015",A2SKHG5ULPYIMC,B0002GFP9I,BellaLatinaReina-ACP Zero-Nine-Five,My son & band teacher loves it. Really nice. B...,Really Helps With Their Practice Pad,1450828800,NaN,NaN,NaN,positive
3,True,"09 4, 2013",A2MFJ6C67AL015,B003AJVKDO,BOS116,This was a gift for a friend who is a fan of t...,Guitar picks,1378252800,NaN,{'Color:': ' Logo'},NaN,positive
4,True,"08 28, 2015",ACYCDUUE9A3T2,B004XNK7AI,jack edwards,Wish it would attach to my violin a little eas...,... to my violin a little easier but it works ...,1440720000,NaN,{'Color:': ' Black - SN-5'},NaN,positive


In [ ]:
# https://stackoverflow.com/questions/34724246/attributeerror-float-object-has-no-attribute-lower

data_text = data_df.reviewText.astype('str')
data_labels = data_df.rating.astype('str')

In [ ]:
pattern = r'[^\w\s]|\d'
data_text = data_text.str.replace(pattern, '')
data_text[0]

<ipython-input-38-09384e363b90>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data_text = data_text.str.replace(pattern, '')


'Have  of these in operation great lights'

In [ ]:
labels = pd.get_dummies(data_labels)
labels

,negative,neutral,positive
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
221387,0,0,1
221388,0,0,1
221389,0,0,1
221390,0,0,1


In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(data_text)
sequences = tokenizer.texts_to_sequences(data_text)

In [ ]:
from keras.utils import pad_sequences

maxlen = 300
padded_data_sequences = pad_sequences(sequences, maxlen=maxlen)

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(padded_data_sequences, labels, test_size=0.2)

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

vocabulary_size = 20000
embedding_size = 300
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=maxlen))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 300)          6000000   
                                                                 
 flatten_3 (Flatten)         (None, 90000)             0         
                                                                 
 dense_3 (Dense)             (None, 3)                 270003    
                                                                 
Total params: 6,270,003
Trainable params: 6,270,003
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(
    train_X,
    train_y,
    epochs=30,
    validation_split=0.2,
    validation_steps=10,
)

Epoch 1/30
1462/4428 [========>.....................] - ETA: 2:42 - loss: 0.3945 - accuracy: 0.8707

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_X, test_y)
print('Accuracy: {}    Loss: {}'.format(test_acc, test_loss))
model_scores.append([model, test_acc])

In [ ]:
!curl http://zacharski.org/files/courses/cs419/glove.6B.100d.zip > glove.6B.100d.zip
!ls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  128M  100  128M    0     0   121M      0  0:00:01  0:00:01 --:--:--  121M
glove.6B.100d.zip  sample_data


In [ ]:
!unzip glove.6B.100d.zip

Archive:  glove.6B.100d.zip
  inflating: glove.6B.100d.txt       


In [ ]:
!curl https://github.com/zacharski/ml-class/raw/master/data/fasttext.zip > fasttext.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


In [ ]:
import numpy as np

In [ ]:

def create_embedding_matrix(file_name, max_words, embedding_dim):
  embeddings_index = {}
  f = open(file_name)
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
  f.close()

  word_index = tokenizer.word_index

  embedding_matrix = np.zeros((max_words, embedding_dim))
  for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
      if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
  return embedding_matrix

In [ ]:
embedding_dim = 300
max_words = 20000
embedding_matrix = create_embedding_matrix('glove.6B.100d.txt', 20000, 300)
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 300, 300)          6000000   
                                                                 
 flatten_7 (Flatten)         (None, 90000)             0         
                                                                 
 dense_10 (Dense)            (None, 64)                5760064   
                                                                 
 dense_11 (Dense)            (None, 3)                 195       
                                                                 
Total params: 11,760,259
Trainable params: 11,760,259
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit(
    train_X,
    train_y,
    epochs=30,
    validation_split=0.2,
    validation_steps=10,
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_X, test_y)
model_scores.append([model, test_acc])

In [ ]:
embedding_dim = 300
max_words = 20000
embedding_matrix = create_embedding_matrix('fasttext.vec', 20000, 300)
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 300, 300)          6000000   
                                                                 
 flatten_8 (Flatten)         (None, 90000)             0         
                                                                 
 dense_12 (Dense)            (None, 64)                5760064   
                                                                 
 dense_13 (Dense)            (None, 3)                 195       
                                                                 
Total params: 11,760,259
Trainable params: 11,760,259
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit(train_X, train_y)

  33/5535 [..............................] - ETA: 11:20 - loss: 0.6402 - acc: 0.8295

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_X, test_y)
model_scores.append([model, test_acc])

In [ ]:
max = 0
best_model = None
for score in model_scores:
  if score[1] > max:
    max = score[1]
    best_model = score[0]

def mapPredictions(val):
  if val == 0:
    return 'positive'
  if val == 1:
    return 'negative'
  if val == 2:
    return 'neutral'

final_test = pd.read_csv("http://34.82.108.37/amazonReviewsTest.csv")
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(final_test)
sequences = tokenizer.texts_to_sequences(final_test)
maxlen = 300
test_padded_sequences = pad_sequences(sequences, maxlen=maxlen)
predictions = best_model.predict(test_padded_sequences)



1/1 [==============================] - 0s 139ms/step


In [ ]:
predictions = np.argmax(predictions, axis=1)
results = []

for prediction in predictions:
  results.append(mapPredictions(prediction))

In [ ]:
with open(r'AmazonReviewsPredictions.txt', 'w') as fp:
    for item in predictions:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')
from google.colab import files
files.download('AmazonReviewsPredictions.txt')

Done


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>